# Advanced

## pg_walinspect

The extension `pg_walinspect` is providing a SQL interface for getting more detailed WAL information.

It exposes the functions:
- `pg_get_wal_record_info` - by specific lsn
- `pg_get_wal_records_info` - by a lsn range
- `pg_get_wal_block_info` - by a lsn range, broken down to data blocks granularity
- `pg_get_wal_stats` - by a lsn range, aggregated to `resource_manager` / `record_type`

## Change And Inspect Current LSN

### CREATE

```SQL
-- Run in pg_admin !!
-- Changing the LSN with create table
DO $$
DECLARE start_state pg_lsn;
DECLARE after_create pg_lsn;
DECLARE start_create_diff text;
BEGIN
	SELECT pg_current_wal_insert_lsn()
	INTO start_state;

	DROP TABLE IF EXISTS a;
	CREATE TABLE a(a int);
	COMMIT; -- Try to ommit

	SELECT pg_current_wal_insert_lsn()
	INTO after_create;

	SELECT pg_size_pretty(after_create - start_state)
	INTO start_create_diff;

	RAISE NOTICE 'Start State LSN: %', start_state;
	RAISE NOTICE 'After Create LSN: %', after_create;
	RAISE NOTICE 'Create - Start DIFF size: %', start_create_diff;
END;
$$
```

### Using `pg_walinspect`

```BASH
psql -c "CREATE EXTENSION pg_walinspect;"

# Get current LSN
CURRENT_LSN=$(psql -t -c "SELECT pg_current_wal_insert_lsn();" | xargs)

# Change it with making changes in the DB
psql -c "DROP TABLE IF EXISTS Z; SELECT * INTO z FROM generate_series(1,10000000);"

# See new information
psql -x -c "SELECT * FROM pg_get_wal_block_info('$CURRENT_LSN', 'FFFFFFFF/FFFFFFFF', false);"
psql << EOM
    SELECT 
        block_id,
        record_type,
        COUNT(*) count 
    FROM (
        SELECT * 
        FROM 
            pg_get_wal_block_info('$CURRENT_LSN', 'FFFFFFFF/FFFFFFFF', false) block_info
            JOIN (SELECT oid FROM pg_class WHERE relname = 'z') rel_info ON block_info.relfilenode = rel_info.oid
    ) sub
    GROUP BY block_id, record_type;
EOM
```

### INSERT

```SQL
-- Run in pg_admin !!
DO $$
DECLARE start_state pg_lsn;
DECLARE after_insert pg_lsn;
DECLARE start_insert_diff text;
DECLARE rec record;
BEGIN
	SELECT pg_current_wal_insert_lsn()
	INTO start_state;

	INSERT INTO a VALUES (1);
	COMMIT;

	SELECT pg_current_wal_insert_lsn()
	INTO after_insert;

	SELECT pg_size_pretty(after_insert - start_state)
	INTO start_insert_diff;

	RAISE NOTICE '------- Summary -------';
	RAISE NOTICE 'Start State LSN: %', start_state;
	RAISE NOTICE 'After Insert LSN: %', after_insert;
	RAISE NOTICE 'Insert - Create DIFF size: %', start_insert_diff;
	RAISE NOTICE 'WAL Records between Start and Insert:';
	FOR rec in 
		SELECT * 
		FROM pg_get_wal_records_info(start_state, after_insert)
	loop
		RAISE NOTICE '------- WAL Record -------';
		RAISE NOTICE 'Start LSN: %', rec.start_lsn;
		RAISE NOTICE 'End LSN: %', rec.end_lsn;
		RAISE NOTICE 'Prev LSN: %', rec.prev_lsn;
		RAISE NOTICE 'Record Type: %', rec.record_type;
		RAISE NOTICE 'Total Record Length In Bytes: %', rec.record_length + rec.main_data_length;
		RAISE NOTICE 'Block Reference: %', rec.block_ref;
		RAISE NOTICE 'Transaction ID: %', rec.xid;
	end loop;
END;
$$
```

#### Block Ref - What Is That?

In [ ]:
psql << EOM
    SELECT pg_relation_filepath FROM pg_relation_filepath('a');
    SELECT * FROM pg_tablespace;
EOM

### CREATE & INSPECT

In [ ]:
-- Run in pg_admin !!
-- What actually happens when we create a new table
DO $$
DECLARE start_state pg_lsn;
DECLARE after_create pg_lsn;
DECLARE start_create_diff text;
DECLARE rec record;
DECLARE rel_name text;
BEGIN
	DROP TABLE IF EXISTS b;
	DROP TABLE IF EXISTS create_wal_records;
	SELECT pg_current_wal_insert_lsn()
	INTO start_state;

	CREATE TABLE b(a int);
	COMMIT;

	SELECT pg_current_wal_insert_lsn()
	INTO after_create;

	-- Change to not temp to look on the rows with simple select
	CREATE TEMP TABLE create_wal_records AS
	SELECT 
		*,
		SPLIT_PART(
			SUBSTRING(block_ref, '[0-9]\/[0-9]+\/[0-9]*'),
			'/',
			3
		)::oid rel_oid
	FROM pg_get_wal_records_info(start_state, after_create);

	SELECT pg_size_pretty(after_create - start_state)
	INTO start_create_diff;
	

	RAISE NOTICE '------- Summary -------';
	RAISE NOTICE 'Start State LSN: %', start_state;
	RAISE NOTICE 'After Create LSN: %', after_create;
	RAISE NOTICE 'Create - Start DIFF size: %', start_create_diff;

	RAISE NOTICE 'WAL Records between Start and Insert:';
	FOR rec in 
		SELECT wal.*, cl.relname 
		FROM 
			create_wal_records wal
			LEFT JOIN pg_class cl ON wal.rel_oid = cl.oid
	loop
		RAISE NOTICE '------- WAL Record -------';
		RAISE NOTICE 'Relation Name: %', rec.relname;
		RAISE NOTICE 'Start LSN: %', rec.start_lsn;
		RAISE NOTICE 'End LSN: %', rec.end_lsn;
		RAISE NOTICE 'Prev LSN: %', rec.prev_lsn;
		RAISE NOTICE 'Record Type: %', rec.record_type;
		RAISE NOTICE 'Total Record Length in bytes: %', rec.record_length + rec.main_data_length;
		RAISE NOTICE 'Block Reference: %', rec.block_ref;
		RAISE NOTICE 'Transaction ID: %', rec.xid;
	end loop;
END;
$$